In [6]:
import cv2
import os
import time
from confluent_kafka import Producer, Consumer
import socket
from ultralytics import YOLO
import cvzone

# Kafka Configuration
producer_conf = {
    'bootstrap.servers': "localhost:9092",  # Replace with your Kafka hostname
    'client.id': socket.gethostname(),
    'compression.type': 'gzip',
    'message.max.bytes': 500000000  # Adjust as needed
}
producer = Producer(producer_conf)

consumer_conf = {
    'bootstrap.servers': "localhost:9092",  # Replace with your Kafka hostname
    'group.id': 'file_group',
    'auto.offset.reset': 'earliest'
}
consumer = Consumer(consumer_conf)

# Directories
capture_dir = "captured_videos"
received_dir = "received_images"
os.makedirs(capture_dir, exist_ok=True)
os.makedirs(received_dir, exist_ok=True)

# Initialize YOLO model
model = YOLO('yolov8n.pt')

In [7]:
def process_with_yolo(file_path):
    """
    Processes the file with YOLO for object detection.
    """
    if file_path.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Process single image
        results = model(file_path, show=True)
        cv2.waitKey(0)
    elif file_path.lower().endswith(('.mp4', '.avi', '.mkv')):  # Process video
        cap = cv2.VideoCapture(file_path)
        prev_frame_time = 0
        new_frame_time = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            new_frame_time = time.time()
            results = model(frame, stream=True)
            for r in results:
                for box in r.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = round(box.conf[0].item(), 2)  # Convert Tensor to scalar and round
                    cls = int(box.cls[0].item())  # Convert Tensor to scalar
                    class_name = model.names[cls]
                    cvzone.putTextRect(frame, f'{class_name} {conf}', (x1, y1))
                    cvzone.cornerRect(frame, (x1, y1, x2 - x1, y2 - y1))

            fps = 1 / (new_frame_time - prev_frame_time)
            prev_frame_time = new_frame_time
            print(f"FPS: {fps}")
            cv2.imshow("YOLO Prediction", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()


In [8]:
def capture_and_send_video(duration=120, topic='files_topic'):
    """
    Captures video from the webcam for a specified duration and sends it to Kafka.
    """
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Unable to access webcam.")
        return

    # Set video frame size
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define codec and create VideoWriter object
    video_filename = os.path.join(capture_dir, f"webcam_video_{int(time.time())}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
    out = cv2.VideoWriter(video_filename, fourcc, 20.0, (width, height))

    start_time = time.time()

    try:
        while time.time() - start_time < duration:
            ret, frame = cap.read()
            if ret:
                # Write frame to video file
                out.write(frame)
                print(f"Recording video...")

            # Optional: Display the video in real-time
            cv2.imshow("Recording", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            time.sleep(0.5)  # Adjust capture frequency
    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        print("Finished recording video.")

    # Send video to Kafka
    with open(video_filename, 'rb') as file:
        file_bytes = file.read()
        producer.produce(topic, key=os.path.basename(video_filename), value=file_bytes)
        print(f"Video {video_filename} sent to topic {topic}.")
        producer.flush()

In [9]:
def receive_files_from_kafka(topic='files_topic'):
    """
    Consumes files from Kafka and processes them with YOLO.
    """
    consumer.subscribe([topic])
    try:
        while True:
            msg = consumer.poll(1.0)
            if msg is None:
                continue
            if msg.error():
                print(f"Consumer error: {msg.error()}")
                continue
            file_name = msg.key().decode('utf-8').split("\\")[-1]
            file_path = os.path.join(received_dir, file_name)
            with open(file_path, 'wb') as file:
                file.write(msg.value())
                print(f"File saved: {file_path}")
            process_with_yolo(file_path)
    except KeyboardInterrupt:
        print("Consumer stopped.")
    finally:
        consumer.close()

In [10]:
# Capture and send video for 2 minutes
capture_and_send_video(duration=12)

# Start consuming files from Kafka
receive_files_from_kafka()

Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Recording video...
Finished recording video.
Video captured_videos\webcam_video_1734748019.mp4 sent to topic files_topic.
File saved: received_images\webcam_video_1734748019.mp4



0: 480x640 1 person, 72.2ms
Speed: 1.0ms preprocess, 72.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 62.7ms
Speed: 2.1ms preprocess, 62.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 61.3ms
Speed: 1.0ms preprocess, 61.3ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)



FPS: 5.764525919977374e-10
FPS: 0.5097742182326969
FPS: 13.55848572324462


0: 480x640 1 person, 59.1ms
Speed: 1.0ms preprocess, 59.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 55.7ms
Speed: 1.0ms preprocess, 55.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 54.2ms
Speed: 1.0ms preprocess, 54.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)



FPS: 11.720876682623118
FPS: 12.879988945016812
FPS: 12.954858955470513


...

FPS: 12.816583958735668
FPS: 12.389096967047509
FPS: 13.20728646776352


0: 480x640 1 person, 1 bottle, 55.7ms
Speed: 1.1ms preprocess, 55.7ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 bottle, 56.2ms
Speed: 1.0ms preprocess, 56.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 62.9ms
Speed: 2.0ms preprocess, 62.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)



FPS: 12.62876705317006
FPS: 12.8327790407015
FPS: 12.521319628746198


0: 480x640 1 person, 58.6ms
Speed: 1.0ms preprocess, 58.6ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 56.7ms
Speed: 1.0ms preprocess, 56.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)


FPS: 12.897653436818686
FPS: 13.20445909401435
Consumer stopped.
